In [1]:
pip install numpy opencv-python scikit-learn matplotlib


Note: you may need to restart the kernel to use updated packages.


In [5]:
import os
import cv2
import random
import numpy as np
from skimage.feature import hog
from sklearn.svm import SVC  # ✅ Import for CPU-based SVM
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report

# 👇 Your dataset paths (update if needed)
dog_dir = r"C:\Users\91636\Downloads\catdogmain\PetImages\Dog"
cat_dir = r"C:\Users\91636\Downloads\catdogmain\PetImages\Cat"

img_size = 64
max_images = 5000
data, labels = [], []

def extract_hog(image):
    return hog(image, pixels_per_cell=(8, 8), cells_per_block=(2, 2),
               orientations=9, feature_vector=True)

def load_images(folder, label):
    files = os.listdir(folder)
    random.shuffle(files)
    count = 0
    for f in files:
        try:
            img = cv2.imread(os.path.join(folder, f))
            img = cv2.resize(img, (img_size, img_size))
            gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
            features = extract_hog(gray)
            data.append(features)
            labels.append(label)
            count += 1
            if count >= max_images:
                break
        except:
            continue

print("🔄 Loading cats...")
load_images(cat_dir, 0)
print("🔄 Loading dogs...")
load_images(dog_dir, 1)

X = np.array(data).astype(np.float32)
y = np.array(labels).astype(np.int32)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

print("🚀 Training SVM on CPU...")
clf = SVC(kernel='linear')  # CPU-based SVM from scikit-learn
clf.fit(X_train, y_train)

y_pred = clf.predict(X_test)

print("✅ Accuracy:", accuracy_score(y_test, y_pred))
print(classification_report(y_test, y_pred, target_names=["Cat", "Dog"]))


🔄 Loading cats...
🔄 Loading dogs...
🚀 Training SVM on CPU...
✅ Accuracy: 0.7145
              precision    recall  f1-score   support

         Cat       0.73      0.69      0.71      1012
         Dog       0.70      0.74      0.72       988

    accuracy                           0.71      2000
   macro avg       0.72      0.71      0.71      2000
weighted avg       0.72      0.71      0.71      2000



In [7]:
# Save the trained model
import joblib
joblib.dump(clf, "svm_cat_dog_model.pkl")
print("💾 Model saved as 'svm_cat_dog_model.pkl'")


💾 Model saved as 'svm_cat_dog_model.pkl'


In [1]:
import gradio as gr
import cv2
import numpy as np
import joblib
from skimage.feature import hog

# Load the trained model
model = joblib.load("svm_cat_dog_model.pkl")
img_size = 64

# Feature extraction (same as used in training)
def extract_hog(image):
    return hog(image, pixels_per_cell=(8, 8), cells_per_block=(2, 2),
               orientations=9, feature_vector=True)

# Prediction function
def predict(image):
    try:
        # Resize and convert to grayscale
        image = cv2.resize(image, (img_size, img_size))
        gray = cv2.cvtColor(image, cv2.COLOR_RGB2GRAY)
        features = extract_hog(gray)
        features = np.array(features).reshape(1, -1).astype(np.float32)
        
        # Predict
        prediction = model.predict(features)[0]
        label = "Dog" if prediction == 1 else "Cat"

        return f"🧠 Prediction: {label}"
    
    except Exception as e:
        return f"❌ Error: {e}"

# Create Gradio interface
interface = gr.Interface(
    fn=predict,
    inputs=gr.Image(type="numpy", label="Upload Cat or Dog Image"),
    outputs=gr.Textbox(label="Prediction"),
    title="🐶 Cat vs Dog Classifier (SVM)",
    description="Upload a 64x64 image of a cat or dog. Model uses HOG features with SVM."
)

# Launch the app
interface.launch()


* Running on local URL:  http://127.0.0.1:7860
* To create a public link, set `share=True` in `launch()`.
